In [1]:
import sys,os
import shutil
import argparse
import numpy as np
import functools

from astropy import wcs
from astropy.io import fits
from astropy.io import ascii
from astropy.coordinates import Angle
from astropy import units as u

# root_dir = 'C:/'

# sys.path.insert(0, root_dir+'UniGe/global_scripts/')
import colors as cl
import read_write_files as rwf

# from matplotlib_config import params
# import matplotlib.pylab as pylab
# pylab.rcParams.update(params)
from matplotlib import pyplot as plt

# import eazy

from PyQt5.QtWidgets import (QWidget, QGridLayout,
                             QPushButton, QApplication)

# import pylab as p
import scipy.ndimage
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore

### suppressing warnings about fits files
import warnings
warnings.filterwarnings('ignore')


import colormaps as cmaps
from colormaps_fresco import viridis_simple as viridis


In [2]:
from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [2]:
### directories and file names

specs_for_Ivan_dir = '../master/data/grizli/MUSE_FRESCO_z5_6_from-Pascal/' # '../FRESCO/ForIvan/'

input_cat_MUSE_LAEs = './eazy_input_cat_MUSE_LAEs.cat' # '../FRESCO/eazy_input_cat_MUSE_LAEs.cat'

eazy_input_cat = './eazy_input_cat.cat' # '../FRESCO/eazy_demo/eazy_input_cat.cat'

In [3]:

def select_MUSE_LAEs():

    ### read catalog
    
    input_cat_values = np.genfromtxt(eazy_input_cat)
    input_cat_info = (open(eazy_input_cat).readlines())[0].split(' ') # keys for the dictionary

    ### select only objects that are in MUSE
    
    files_specs_for_Ivan_dir = os.listdir(specs_for_Ivan_dir)
    specs_for_Ivan_IDs = np.asarray(list(set(np.asarray([int(i.split('5.1_')[1].split('.')[0]) for i in files_specs_for_Ivan_dir if "beams" in i])))) # cut out the IDs from the file names
    
    band_data = np.genfromtxt(eazy_input_cat)
    band_data_id = np.asarray([i[0] for i in band_data if i[0]==i[0]])
    band_data_ra = np.asarray([i[1] for i in band_data if i[1]==i[1]])
    band_data_dec = np.asarray([i[2] for i in band_data if i[2]==i[2]])

    here_same_arr = []
    ID_phot_cat = []
    for i in range(len(specs_for_Ivan_IDs)):
        here_ID = int(sorted(specs_for_Ivan_IDs)[i])
        file_here = specs_for_Ivan_dir+'gds-grizli-v5.1_{:05d}'.format(here_ID)+'.beams.fits'

        hdu = fits.open(file_here)
        header = hdu[0].header
        hdu.close()

        ra = header['RA']
        dec = header['DEC']

        diff = np.sqrt((band_data_ra-ra)**2+(band_data_dec-dec)**2)
        here_same = np.argmin(diff)
        here_same_arr.append(here_same)
        ID_phot_cat.append(int(band_data_id[here_same]))

    input_cat_MUSE_LAEs_open = open(input_cat_MUSE_LAEs,'w')
    for i in range(len(specs_for_Ivan_IDs)):
        if i == 0:
            input_cat_MUSE_LAEs_open.write(open(eazy_input_cat).readlines()[0][:-3]+' z_spec\n')

        here_ID = int(sorted(specs_for_Ivan_IDs)[i])
        same = here_same_arr[i]+1 # correct offset because first line isn't an object
        line_write = ''
        for j in range(len(input_cat_values[same])):
            if j==0:
                #line_write+=str(int(input_cat_values[same][j]))
                line_write+=str(int(here_ID))
            else:
                line_write+='\t'+str(input_cat_values[same][j])
        line_write+='\t -99.'
        line_write+='\n'
        input_cat_MUSE_LAEs_open.write(line_write)
    return ID_phot_cat


ID_phot_cat = select_MUSE_LAEs()

input_cat_values = np.genfromtxt(input_cat_MUSE_LAEs)
input_cat_info = (open(input_cat_MUSE_LAEs).readlines())[0].split(' ') # keys for the dictionary

input_cat_dict = {}
input_cat_keys = []
for i in range(len(input_cat_info)):
    key = input_cat_info[i].strip('\n')
    input_cat_keys.append(key)
    if key == 'id':
        input_cat_dict[key] = np.asarray([input_cat_values[k+1][i] for k in range(len(input_cat_values)-1)],dtype=int) # excluding the first nan in the first line
    else:
        input_cat_dict[key] = np.asarray([input_cat_values[k+1][i] for k in range(len(input_cat_values)-1)]) # excluding the first nan in the first line

ID = np.asarray(input_cat_dict['id'],dtype=int)
total_num_obj = len(ID)

input_cat_keys.append('SFR')
input_cat_keys.append('mass')
input_cat_keys.append('chi2')

input_cat_dict['SFR'] = np.asarray([-99.0 for i in range(len(ID))])
input_cat_dict['mass'] = np.asarray([-99.0 for i in range(len(ID))])
input_cat_dict['chi2'] = np.asarray([-99.0 for i in range(len(ID))])

In [5]:

### Test plots

test_ID = '02664'

def read_spec_1D(infile):

    hdu = fits.open(infile)
    header = hdu[1].header
    data = hdu[1].data 
    hdu.close()

    wave = np.asarray([i[0] for i in data])
    flux = np.asarray([i[1] for i in data])
    error = np.asarray([i[2] for i in data])

    infos_dict = {'wave':wave,
                  'flux':flux,
                  'error':error}

    return infos_dict,header

def plot_spec():
    infos_dict,header = read_spec_1D(spec1)
    plt.plot(infos_dict['wave'],infos_dict['flux'],'b-')
    plt.plot(infos_dict['wave'],infos_dict['error'],'r-',alpha=0.5)
    #plt.plot(infos_dict['wave'],infos_dict['contam'],'m-',alpha=0.5)
    #plt.plot(infos_dict['wave'],infos_dict['trace'],'k-',alpha=0.5)
    #plt.plot(infos_dict['wave'],infos_dict['sensitivity'],'y-',alpha=0.5)
    plt.show()


spec1 = specs_for_Ivan_dir+"gds-grizli-v5.1_"+test_ID+".1D.fits"
infos_dict_spec,header = read_spec_1D(spec1)

#plot_spec()

def read_spec_2D(infile):
    hdu = fits.open(infile)
    image_header = hdu[0].header
    header = hdu[1].header
    data           = hdu[1].data 
    hdu.close()
    return header,data

spec2 = specs_for_Ivan_dir+'gds-grizli-v5.1_'+test_ID+'.stack.fits'

def plot_all():
    header,data = read_spec_2D(spec2)
    plt.imshow(data)
    plt.show()
#plot_all()

def plot_cutout(infile):
    hdu = fits.open(infile)
    header_band = hdu[1].header
    data_band = np.asarray(hdu[1].data) 

    hdu.close()

    #band_ps = pix_scale(header_band)

    plt.imshow(data_band,origin='lower')
    
#plot_cutout(specs_for_Ivan_dir+'gds-grizli-v5.1_'+str(test_ID)+'.beams.fits')    



In [4]:
### functions

def write_output(ID,comments,new_catalog):
    # write identifications in fits file 

    # make sure that the comments don't get truncated
    max_len = 0
    for comment in comments:
        if len(comment) > max_len:
            max_len = len(comment)
    max_len += 1

    columns = []
    for column in input_cat_keys:
        dtype = type(input_cat_dict[column][0])

        if dtype is np.float64:
            format="D"
        elif dtype is np.int32:
            format="J"
        elif dtype is str:
            format="20A"
        else:
            format="10A"
        c = fits.Column(name=column, format=format, array=input_cat_dict[column])
        columns.append(c)

    c = fits.Column(name='comments', format=str(max_len)+"A", array=comments)
    columns.append(c)

    table_hdu = fits.BinTableHDU.from_columns(columns)
    table_hdu.writeto(new_catalog,overwrite=True)

def read_hdu(infile,hdunum,nans_to_value=False,nanvalue=0,memmap=True):
    """
    image_data, header = read_hdu(infile,hdunum,nans_to_value=False,nanvalue=0)
    Read in a HDU <hdunum> from FITS-File <infile>, returns the
    data <image_data> and the header <header>.
    Additionally with nans_to_value = True (default: False) one can set
    all NaNs to a specific <nanvalue> (default: 0)

    """
    hdu = fits.open(infile,memmap=memmap)
    header = hdu[hdunum].header
    image_data = hdu[hdunum].data 
    hdu.close()
    if nans_to_value == True:
        # change NaNs to nanvalue
        select = np.isnan(image_data)
        image_data[select] = nanvalue
    return image_data,header

def vminvmax(image, scale=0.99, nans=0):
    """
    vmin,vmax = vminvmax(data, scale=0.99, nans=0)

    In:
    ---
    data - 2D array (intensity image)
    scale (=0.99) - scale factor for colorbar
    nans (=0) value with wich NaNs in the array are replaced

    Out:
    ---
    vmin, vmax -- values to be used in matplotlib.imshow as vmin & vmax
    """
    assert scale <= 1 and scale > 0 
    data = image.copy()    

    data[np.isnan(data)] = nans
    if len(data.shape) > 1:
        data = data.flatten()
    data = np.sort(data)

    length = len(data)

    vmax_index = int(np.floor(scale*length))
    vmin_index = int(np.ceil((1-scale)*length))

    vmin = data[vmin_index]
    vmax = data[vmax_index]

    return vmin, vmax

def pix_scale(header):
    """
    Prints out averaged pixel scale in arcseconds at reference pixel. 
    Ignores dissortions, but this OK considering our small field of view.

    In:
    ---
    header ... an astropy.io.fits header object

    Out:
    ---
    pix_scale ... average linear extent of a pixel in arc-seconds
    """

    wcs_obj = wcs.WCS(header,relax=False)
    scale_deg_xy = wcs.utils.proj_plane_pixel_scales(wcs_obj)
    scale_deg = p.sum(scale_deg_xy[:2]) / 2.

    scale_deg = Angle(scale_deg,unit=u.deg)
    scale_asec = scale_deg.arcsec

    return scale_asec

def radec_to_pix(ra_coords,dec_coords,header,origin=0):
    """
    Converts RA & DEC world coordinates to pixel coordinates.

    In:
    ---
    ra_coords ... 1D array of RA in degrees (float)
    dec_coords ... 1D array of corresponding DEC in degrees (float)
    header ... an astropy.io.fits header object
    origin ... output coordinates 0-indexed if 0, or 1-indexed if 1
               (default: 0)

    Out:
    ---
    x_coords, y_coords ... 2-tuple of 1D arrays with pixel coordinates
    """

    wcs_obj = wcs.WCS(header, relax=False)  # no HST only spec
                                            # allowed, only WCS
                                            # standard
    coords = wcs_obj.wcs_world2pix(ra_coords,dec_coords,origin)
    x_coords = coords[0]
    y_coords = coords[1]
    return x_coords, y_coords


In [6]:
### initialise 

viridis_lookuptable = np.asarray([np.asarray(cmaps.viridis(k))*255 for k in range(200)])

# initialise values for cuts in images
cuts_cutout = [2000]
cuts_spec_2D = [400]
cuts_spec_2D_model = [400]

cut_cutout1 = [0]
cut_cutout2 = [100]

z_slider = [4.6]

### initiate lists and variables

comments = np.asarray(['-' for i in range(len(ID))])
# This makes sure that if the comments were restricted to some length before,
# this restriction is now lifted
lines_comment_new = ['-' for a in comments]
lines_comment_new[:] = comments
comments = lines_comment_new

j = 0 # initialise running variable for objects
width_window = [8.]  # width of the thumbnail cut-out windows in arcsec  

### set colours
c_data_crossline = 'r'

In [ ]:
### prepare eazy

from pathlib import Path
import tempfile

def symlink(target, link_name, overwrite=False):

    #Create a symbolic link named link_name pointing to target.
    #If link_name exists then FileExistsError is raised, unless overwrite=True.
    #When trying to overwrite a directory, IsADirectoryError is raised.


    if not overwrite:
        os.symlink(target, link_name)
        return

    # os.replace() may fail if files are on different filesystems
    link_dir = os.path.dirname(link_name)

    # Create link to target with temporary filename
    while True:
        temp_link_name = tempfile.mktemp(dir=link_dir)

        # os.* functions mimic as closely as possible system functions
        # The POSIX symlink() returns EEXIST if link_name already exists
        # https://pubs.opengroup.org/onlinepubs/9699919799/functions/symlink.html
        try:
            os.symlink(target, temp_link_name)
            break
        except FileExistsError:
            pass

    # Replace link_name with temp_link_name
    try:
        # Pre-empt os.replace on a directory with a nicer message
        if not os.path.islink(link_name) and os.path.isdir(link_name):
            raise IsADirectoryError(f"Cannot symlink over existing directory: '{link_name}'")
        os.replace(temp_link_name, link_name)
    except:
        if os.path.islink(temp_link_name):
            os.remove(temp_link_name)
        raise

def find_python_path(key):

    out_path = []
    for path in sys.path:
        if key in path:
            out_path.append(Path(path))

    assert len(out_path) > 0, f'Did not find path containing key <{key}>, make sure it is in your PYTHONPATH environment variable'
    assert len(out_path) < 2, f'More than one path in your PYTHONPATH environment variable contains the key <{key}>, do not know which one to use'

    return out_path[0]


eazy_PATH = 'C:\RUG\FRESCO\eazy-py'
import sys
if not eazy_PATH in sys.path:
    sys.path.append(eazy_PATH)
eazy_path = find_python_path('eazy-py')

eazy_output_path = Path('.')
templ_link = eazy_output_path / 'templates'

if not os.path.exists('./temp'):
    symlink(eazy_path / 'eazy' / 'data' / 'templates', Path('./temp'), overwrite=True)

if os.path.exists('./FILTER.RES.latest'):
    os.remove('./FILTER.RES.latest')

symlink(f'{eazy_path}/eazy-photoz/filters/FILTER.RES.latest', './FILTER.RES.latest', overwrite=True)


translate_file = f'./eazy.translate'

config_params = {
                "TEMPLATES_FILE" : "./temp/sfhz/carnall_sfhz_13.param",
                "WAVELENGTH_FILE" : f"./temp/uvista_nmf/lambda.def",
                "TEMP_ERR_FILE" : f"./temp/template_error_cosmos2020.txt",
                "TEMP_ERR_A2" : 1.0,
                "SYS_ERR" : 0,
                "APPLY_IGM" : "y",
                "CAT_HAS_EXTCORR" : True,
                "N_MIN_COLORS" : 5.0,
                "APPLY_PRIOR" : "n",
                "PRIOR_FILE" : f"./temp/prior_F160W_TAO.dat",
                "PRIOR_FILTER" : 377, #prior is disabled ("APPLY_PRIOR" : "n"), but this is used by the iterative zpt. optimization below
                                    #377 refers to the F444W filter
                "PRIOR_ABZP" : 31.4,
                "PRIOR_FLOOR" : 0.01,
                "Z_MIN" : 0,
                "Z_MAX" : 18,
                "Z_STEP" : 0.01,
                "REST_FILTERS" : "---",
                "H0" : 70.0,
                "OMEGA_M" : 0.3,
                "OMEGA_L" : 0.7,
                "CATALOG_FILE" : f'./'+input_cat_MUSE_LAEs,
                "CATALOG_FORMAT" : "ascii",
                "OUTPUT_DIRECTORY" : ".",
                "MAIN_OUTPUT_FILE" : eazy_output_path / 'eazy_output',
                "FIX_ZSPEC" : True
            }

In [7]:
# in vacuum, most are taken from (not above 2000 Angstrom)
# https://physics.nist.gov/PhysRefData/ASD/lines_form.html
# some are from http://classic.sdss.org/dr4/algorithms/linestable.html

lbda_full = {
    'OIV 1': 1031.93,
    'OIV 2': 1037.62,
    'Lya': 1215.67,
    'NV': 1240.81,
    'OI': 1305.53,
    'SiIV 1': 1393.76,
    'SiIV 2': 1402.77,
    'CIV 1': 1548.195, #'Morton1991tab5'
    'CIV 2': 1550.770, #'Morton1991tab5'
    'HeII': 1640.42,
    'OIII 4': 1666.15,
    'CIII 1': 1906.683,
    'CIII 2': 1908.734,
    'CII': 2326.0,
    'FeII A1': 2344.21,
    'FeII A2': 2374.46,
    'FeII A3': 2382.76,
    'FeII* 1': 2396.36,
    'FeII A4': 2586.65,
    'FeII A5': 2600.17,
    'FeII* 2': 2612.65,
    'FeII* 3': 2626.45,
    'MgII 1': 2796.35,
    'MgII 2': 2803.53,
    'OII 1': 3727.092,
    'OII': 3729.875,
    'NeIII 2': 3869.81,
    'HeI': 3888.65,
    'H8': 3890.15,
    'CaII A1': 3934.777,
    'NeIII 1': 3968.53,
    'CaII A2': 3969.588,
    'Hd': 4102.89,
    'Hg': 4341.68,
    'OIII 3': 4363.21,
    'Hb': 4862.68,
    'OIII 1': 4960.295,
    'OIII 2': 5008.240,
    'HeI 2': 5875.68,
    'OI 1': 6300.3,
    'OI 2': 6364.83,
    'NII 1': 6549.86,
    'Ha': 6564.61,
    'NII 2': 6585.27,
    'SII 1': 6718.29,
    'SII 2': 6732.67,
    'ArIII 1': 7136.30
}

lbda = {
    'Lya': 1215.67,
    'CIV 1': 1548.195, #'Morton1991tab5'
    'CIV 2': 1550.770, #'Morton1991tab5'
    'HeII': 1640.42,
    'CIII 1': 1906.683,
    'CIII 2': 1908.734,
    'CII': 2326.0,
    'MgII 1': 2796.35,
    'MgII 2': 2803.53,
    'OII 1': 3727.092,
    'OII': 3729.875,
    'OIII 3': 4363.21,
    'Hb': 4862.68,
    'OIII 1': 4960.295,
    'OIII 2': 5008.240,
    'Ha': 6564.61,
    'SII 1': 6718.29,
    'SII 2': 6732.67,
}



In [ ]:
### Run eazy with all objects

input_cat_MUSE_LAEs_read = ascii.read(input_cat_MUSE_LAEs) #eazy_input_cat) 
        
#get correct MW extinction
# TODO: Gives error, needs check
if not 'MW_EBV' in config_params.keys():
    config_params['MW_EBV'] = eazy.utils.get_irsa_dust(np.nanmedian(input_cat_MUSE_LAEs_read['ra']), np.nanmedian(input_cat_MUSE_LAEs_read['dec']))

#use built-in <iterate_zp_templates> function to optimize/correct for zeropoint offsets
optimize_zpt_iter = 3 #nr of iterations
nr_of_filters = 13 #nr of available filters

with suppress_stdout():
    eazy_inst = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, zeropoint_file=None, 
                                   params=config_params, load_prior=False,load_products=False,n_proc=8)

    with suppress_stdout():
        if optimize_zpt_iter > 0:
            n_bin = np.minimum(eazy_inst.NOBJ//100, 180)
            for i in range(optimize_zpt_iter):
                sn = eazy_inst.fnu/eazy_inst.efnu
                clip = np.sum(sn > 1, axis=1) > np.min([nr_of_filters, 5]) # Generally make this higher to ensure reasonable fits
                eazy_inst.iterate_zp_templates(idx=eazy_inst.idx[clip], update_templates=False, update_zeropoints=True, iter=i, n_proc=8, 
                                                    save_templates=False, error_residuals=False, NBIN=n_bin, get_spatial_offset=False)
                plt.close('all')

    #actual fit
    with suppress_stdout():
        eazy_inst.fit_catalog()

    #get output
    with suppress_stdout():     
        zout, _ = eazy_inst.standard_output()   

    ### move all eazy output in directory
    here_files = os.listdir('.')
    for file in here_files:
        if 'output' in file and not os.path.isdir(file):
            shutil.move(file, 'eazy_output/'+file)



100%|██████████| 14/14 [00:11<00:00,  1.17it/s]
295it [00:00, 10732.33it/s]
100%|██████████| 42/42 [00:00<00:00, 182.36it/s]


In [8]:
'''
### playing around with fitting objects with eazy

import time 

start = time.time()

print('Done with eazy_inst')

#actual fit
with suppress_stdout():
    eazy_inst.fit_catalog(idx = np.asarray([0]))

print('Done fitting.')

#get output
with suppress_stdout():     
    zout, _ = eazy_inst.standard_output()   

### move all eazy output in directory
here_files = os.listdir('.')
for file in here_files:
    if 'output' in file and not os.path.isdir(file):
        shutil.move(file, 'eazy_output/'+file)

with suppress_stdout(): 
    eazy_fig, eazy_ax = eazy_inst.show_fit(zout['id'][0], show_fnu=1, show_components=False)
    plt.close('all')

print(zout['z_spec'][0])

ind = 0
eazy_id = str(eazy_inst.OBJID[ind])
eazy_z = eazy_inst.zbest[ind]

stop = time.time()

print('This took',stop-start,'seconds.')

print('\n Eazy redshift',eazy_z)

print(zout.keys())
'''

"\n### playing around with fitting objects with eazy\n\nimport time \n\nstart = time.time()\n\nprint('Done with eazy_inst')\n\n#actual fit\nwith suppress_stdout():\n    eazy_inst.fit_catalog(idx = np.asarray([0]))\n\nprint('Done fitting.')\n\n#get output\nwith suppress_stdout():     \n    zout, _ = eazy_inst.standard_output()   \n\n### move all eazy output in directory\nhere_files = os.listdir('.')\nfor file in here_files:\n    if 'output' in file and not os.path.isdir(file):\n        shutil.move(file, 'eazy_output/'+file)\n\nwith suppress_stdout(): \n    eazy_fig, eazy_ax = eazy_inst.show_fit(zout['id'][0], show_fnu=1, show_components=False)\n    plt.close('all')\n\nprint(zout['z_spec'][0])\n\nind = 0\neazy_id = str(eazy_inst.OBJID[ind])\neazy_z = eazy_inst.zbest[ind]\n\nstop = time.time()\n\nprint('This took',stop-start,'seconds.')\n\nprint('\n Eazy redshift',eazy_z)\n\nprint(zout.keys())\n"

In [11]:
# the default would be a black background and white plot for pyqtgraph
pg.setConfigOption('background', 'w')
pg.setConfigOption('foreground', 'k')


class main_Window(QtGui.QMainWindow):

    signal_j = QtCore.pyqtSignal(int)

    def __init__(self):
        super(main_Window, self).__init__()
        self.initUI()

        
    def initUI(self):

        self.j = j
        self.ID_here = ID[self.j]

        ### status bar
        self.statusBar().showMessage('Message in statusbar.')


        if self.j == 0:
            self.main_GUI = FRESCO(self,self.j,self.statusBar())
            #self.main_GUI.signal1.connect(self.show_status)
            self.setCentralWidget(self.main_GUI)
            #self.statusBar().showMessage('') 

        ### size and logo
        self.setGeometry(600, 500, 2550, 1450)  # position and size of the window
        self.setWindowTitle('FRESCO')   # title of the window
        self.setWindowIcon(QtGui.QIcon('logo2_2.png'))  # logo in upper left corner
        self.show()

        self.main_GUI.signal_next_previous[str].connect(self.show_object)

    ####################################################################

    def show_object(self,text): # Goes to next or previous object

        max_len = len(ID)

        ID_old = ID[self.j]

        if text == 'next':
            forward_back = 1
        elif text == 'previous':
            forward_back = -1
        else: 
            forward_back = 0
            
        if self.j == max_len-1: # if the end of the lines is reached
            # stay at this ID (the last)
            ID_new = ID_old
            # if we are going back, it's fine, we go back
            if text=='previous':
                ID_new = ID[self.j]
            # if we are going forward,go to first object
            if text=='next':
                print('Went too far!')
                #self.went_through()
        elif self.j < max_len-1 and self.j>=0:
            ID_new = ID[self.j+forward_back]
        else: # if you go back from the first object, to go the last
            ID_new = ID[0]

        self.j+=forward_back

        if self.j<0:
            self.j = max_len-1
        if self.j >= max_len:
            self.went_through()

        self.main_GUI = FRESCO(self,self.j,self.statusBar())
        self.setCentralWidget(self.main_GUI)
        self.main_GUI.signal_next_previous[str].connect(self.show_object)
        self.signal_j.emit(self.j) 
        #self.main_GUI.plot()
        #self.main_GUI.send_message()



############################################################################################################
### Main GUI

class FRESCO(QWidget):

    signal_next_previous = QtCore.pyqtSignal(str)
    #signal_previous = QtCore.pyqtSignal(str)

    def __init__(self,parent,j,status):
        super().__init__()
        #self.setStyleSheet("background-color: black;")
        self.initUI(j,status)

    def initUI(self,j,status):

        ### initialise
        self.j = j
        self.status = status
        message = 'None'
        self.message = message
        self.z_slider = z_slider[0]

        ### general layout of the window
        grid = QtGui.QGridLayout()
        grid.setSpacing(10)
        self.setLayout(grid)

        ### write ID as title and reset button
        self.ID_here = ID[self.j]
        reset_button = QPushButton('ID  '+str(int(self.ID_here)))
        reset_button.clicked.connect(self.np_obj)
        reset_button.setToolTip('Reset plots')
        grid.addWidget(reset_button,1,10,1,2)
        number_objs_label = QtGui.QLabel(' (#'+str(self.j+1)+' of '+str(total_num_obj)+' objects)', self)
        grid.addWidget(number_objs_label,1,12,1,1)

        ### Close button
        # TODO
        #close_button = QPushButton('Close')
        #close_button.clicked.connect(self.close_prog)
        #close_button.setToolTip('Close the program.')
        #grid.addWidget(close_button, 1,31,1,1)

        ### image cutout
        self.cutout = pg.GraphicsLayoutWidget(self)
        cutout_size = 500
        self.cutout.setMinimumSize(cutout_size,cutout_size)
        self.cutout.setMaximumSize(cutout_size,cutout_size)
        grid.addWidget(self.cutout,4,1,5,5)

        cutout_title = ('Image:')
        cutout_title_label = QtGui.QLabel(cutout_title, self)
        grid.addWidget(cutout_title_label,2,1,1,1)
    
        # cuts for the image
        self.cut_cutout1 = cut_cutout1[0] #np.round(cut_cutout1[0],6)
        self.cut_cutout2 = cut_cutout2[0] #np.round(cut_cutout2[0],6)
    
        # enter cuts
        cut_cutout1_comm = QtGui.QLabel('Cut min', self)
        grid.addWidget(cut_cutout1_comm,3,1,1,1)
        cut_cutout2_comm = QtGui.QLabel('Cut max', self)
        grid.addWidget(cut_cutout2_comm,3,3,1,1)
    
        self.le_cut_cutout1 = QtGui.QLineEdit(self)
        grid.addWidget(self.le_cut_cutout1,3,2,1,1)
        self.le_cut_cutout1.returnPressed.connect(self.show_cutout)
        self.le_cut_cutout2 = QtGui.QLineEdit(self)
        grid.addWidget(self.le_cut_cutout2,3,4,1,1)
        self.le_cut_cutout2.returnPressed.connect(self.show_cutout)

        empty = QtGui.QLabel('                                                                                                     ', self)
        grid.addWidget(empty,3,5,1,1)
    
        self.le_cut_cutout1.setText(str(self.cut_cutout1))
        self.le_cut_cutout2.setText(str(self.cut_cutout2))

        self.show_cutout()
        
        ### 2D spec
        self.spec_2D = pg.GraphicsLayoutWidget(self)
        spec_2D_size = 500
        grid.addWidget(self.spec_2D,9,2,2,8)

        spec_2D_show = [True]
        if spec_2D_show[0]:  
            self.show_spec_2D()

        # set slider to change cuts in 2D spec
        self.cuts_spec_2D = cuts_spec_2D[0]
        sld_spec_2D = QtGui.QSlider(QtCore.Qt.Vertical, self)
        grid.addWidget(sld_spec_2D,9,1,2,1)
        sld_spec_2D.setRange(10,1000)
        sld_spec_2D.setValue(self.cuts_spec_2D)
        sld_spec_2D.setSingleStep(1)
        sld_spec_2D.valueChanged[int].connect(self.changeValue_spec_2D_cuts)
        sld_spec_2D.setToolTip('Slide to change cuts.')   

        ### set full spectrum
        self.full_spec = pg.GraphicsLayoutWidget(self)
        self.full_spec.setMinimumWidth(1000)
        grid.addWidget(self.full_spec,12,2,4,8)
        self.plot_spec()

        # set redshift slider
        sld_redshift = QtGui.QSlider(QtCore.Qt.Horizontal, self)
        grid.addWidget(sld_redshift,18,1,1,8)
        sld_redshift.setRange(z_slider[0]*1000,10000) # large numbers to allow for smaller steps
        sld_redshift.setValue(z_slider[0]*1000)
        sld_redshift.setSingleStep(1)
        sld_redshift.valueChanged[int].connect(self.changeValue_sld_redshift)
        sld_redshift.setToolTip('Slide to redshift.')   

        title_z = QtGui.QLabel('z = ', self)
        self.le_z = QtGui.QLineEdit(self)
        grid.addWidget(title_z,18,10,1,1)
        grid.addWidget(self.le_z,18,11,1,1)

        #z_label = QtGui.QLabel('z = ', self)
        #z_label2 = QtGui.QLabel(str(self.z_slider), self)
        #grid.addWidget(z_label,19,10,1,1)
        #grid.addWidget(z_label2,19,11,1,1)

        ### set buttons for next or previous object
        np_name = {0:['next','right'],
                   1:['previous','left']}
        for i in range(2):
            button_np = QtGui.QPushButton('', self)
            button_np.setIcon(QtGui.QIcon(np_name[i][0]+'.png'))
            button_np.setToolTip('Look at '+np_name[i][0]+' object.')
            button_np.setText(np_name[i][0])
            button_np.clicked.connect(self.np_obj)
            button_np.setShortcut(np_name[i][1])
            if np_name[i][0] == 'previous':
                grid.addWidget(button_np,2,17,1,1)
            else:
                grid.addWidget(button_np,2,19,1,1)

        ### comment line
        title_comm = QtGui.QLabel('Comment:', self)
        self.le_comment = QtGui.QLineEdit(self)
        grid.addWidget(title_comm,3,12,1,1)
        grid.addWidget(self.le_comment,4,12,1,3)
    
        ### Write position (convert to hour)
        RA = input_cat_dict['ra'][self.j]
        DEC = input_cat_dict['dec'][self.j]
        RA_hour = Angle(RA,unit=u.degree).hour
        RA_hour1 = int(RA_hour)
        RA_hour2 = 60*(RA_hour-RA_hour1)
        RA_hour3 = np.round(60*(RA_hour2-int(RA_hour2)),5)
        RA_label1 = QtGui.QLabel('RA:', self)
        RA_label2 = QtGui.QLabel(str(RA_hour1)+':'+
                                str(int(RA_hour2))+':'+
                                str(RA_hour3), self)
        grid.addWidget(RA_label1,4,17,1,1)
        grid.addWidget(RA_label2,4,18,1,1)
        DEC_hour = DEC
        DEC_hour1 = int(DEC)
        DEC_hour2 = -60*(DEC_hour-DEC_hour1)
        DEC_hour3 = np.round(60*(DEC_hour2-int(DEC_hour2)),5)
        DEC_label1 = QtGui.QLabel('DEC:', self)
        DEC_label2 = QtGui.QLabel(str(DEC_hour1)+':'+
                                 str(int(DEC_hour2))+':'+
                                 str(DEC_hour3), self)
        grid.addWidget(DEC_label1,5,17,1,2)
        grid.addWidget(DEC_label2,5,18,1,2)

        ### eazy results
        self.eazy_fig_widget = pg.GraphicsLayoutWidget(self)
        self.eazy_z_widget = pg.GraphicsLayoutWidget(self)
        grid.addWidget(self.eazy_fig_widget,7,12,5,10)
        grid.addWidget(self.eazy_z_widget,12,12,5,7)
        self.show_eazy_fig()

        ### Write eazy results

        '''
        z_raw_chi2 = np.round(self.zout['z_raw_chi2'][self.j],3)
        eazy_raw_chi2 = QtGui.QLabel('Chi2: '+str(z_raw_chi2), self)
        grid.addWidget(eazy_raw_chi2,7,31,1,1)

        raw_chi2 = np.round(self.zout['raw_chi2'][self.j],3)
        eazy_raw_chi2 = QtGui.QLabel('Chi2: '+str(raw_chi2), self)
        grid.addWidget(eazy_raw_chi2,8,31,1,1)
        '''

        self.z_phot_chi2 = np.round(self.zout['z_phot_chi2'][self.j],3)
        eazy_raw_chi2 = QtGui.QLabel('Chi2: '+str(self.z_phot_chi2), self)
        grid.addWidget(eazy_raw_chi2,9,31,1,1)

        self.sfr = np.round(self.zout['sfr'][self.j],3)
        eazy_sfr = QtGui.QLabel('SFR: '+str(self.sfr), self)
        grid.addWidget(eazy_sfr,10,31,1,1)

        self.mass = np.round(self.zout['mass'][self.j]/10**9,3)
        eazy_mass = QtGui.QLabel('mass: '+str(self.mass), self)
        grid.addWidget(eazy_mass,11,31,1,1)

        


    #############################################################################
    ### functions

    def close_prog(self):
        # TODO: save catalogue!
        print('Well done! :)')
        #self.close()
      
    def changeValue_cutout_cuts(self,value):
        self.cuts_cutout = value
        cuts_cutout[0] = value
        self.show_cutout()

    def show_spec_2D(self):

        self.spec_2D.clear()

        hdu = fits.open(specs_for_Ivan_dir+'gds-grizli-v5.0_0'+str(self.ID_here)+'.stack.fits')
        image_header = hdu[0].header
        header = hdu[1].header
        image_sci = hdu[1].data 
        hdu.close()

        img_spec_2D = pg.ImageItem(border='k')
        viridis_lookuptable = np.asarray([np.asarray(cmaps.viridis(k))*255 for k in range(cuts_spec_2D[0])])
        img_spec_2D.setLookupTable(viridis_lookuptable)
        img_spec_2D.setImage(np.rot90(image_sci)[::-1])
        self.view_spec_2D = self.spec_2D.addViewBox()
        self.view_spec_2D.addItem(img_spec_2D)
        self.view_spec_2D.setAspectLocked(True) 

    def changeValue_spec_2D_cuts(self,value):
        self.cuts_spec_2D = value
        cuts_spec_2D[0] = value
        self.show_spec_2D()

    def changeValue_sld_redshift(self,value):
        self.z_slider = float(value/1000)
        #z_slider[0] = value
        self.le_z.setText(str(self.z_slider))
        self.plot_spec()

        input_cat_dict['z_spec'][self.j] = self.z_slider

    def changeValue_spec_2D_model_cuts(self,value):
        self.cuts_spec_2D_model = value
        cuts_spec_2D_model[0] = value
        self.show_spec_2D_model()

    def plot_spec(self):

        self.full_spec.clear()
        pfs = self.full_spec.addPlot()
        zline = pg.InfiniteLine(angle=0, movable=False, pen=(50, 100, 100))

        spec1 = specs_for_Ivan_dir+"gds-grizli-v5.0_0"+str(self.ID_here)+".1D.fits"
        infos_dict_spec,header = read_spec_1D(spec1)    

        lines_Pen2 = pg.mkPen(color='r', width=4, alpha = 0.7)
        pfs.plot(infos_dict_spec['wave'],infos_dict_spec['flux'], pen='k')
        pfs.plot(infos_dict_spec['wave'],infos_dict_spec['error'], pen='r')
        pfs.addItem(zline,ignoreBounds=True)
        #pfs.setYRange(-yrange[0]/4., yrange[0]*(3./4.))
        pfs.setYRange(-0.08,0.18)
        pfs.setXRange(infos_dict_spec['wave'][0],infos_dict_spec['wave'][-1])


        wave_unit = header['TUNIT1']
        flux_unit = header['TUNIT2']
        styles = {'color':'r', 'font-size':'20px'}
        pfs.setLabel('bottom',wave_unit, **styles)
        pfs.setLabel('left',flux_unit, **styles)

        
        # lines for all other lines
        c_otherlines = np.asarray(cl.viridis_more[9])*255
        lines_Pen = pg.mkPen(color=c_otherlines, width=1)
        for j in lbda:
            vLines_all = pg.InfiniteLine(angle=90, movable=False, 
                                         pen=lines_Pen)
            line_pos = lbda[j]*(self.z_slider+1)
            vLines_all.setPos(line_pos)
            pfs.addItem(vLines_all,ignoreBounds=True)
            line_name_text = j
            annotate2 = pg.TextItem(text=line_name_text,color=c_otherlines,
                                    anchor=(1,1),angle=-90)
            annotate2.setPos(line_pos,0.1)
            pfs.addItem(annotate2)    
        
    def np_obj(self,parent):
        # go to next or previous object in catalogue

        # Save comment
        comment = self.le_comment.text()
        if len(comment)!=0:
            comments[self.j] = comment

        self.cut_cutout1 = float(self.le_cut_cutout1.text())
        self.cut_cutout2 = float(self.le_cut_cutout2.text())
        cut_cutout1[0] = self.cut_cutout1
        cut_cutout2[0] = self.cut_cutout2

        self.save_now()
        self.signal_next_previous.emit(self.sender().text()) 

    def save_now(self):
        input_cat_dict['SFR'][self.j] = self.sfr
        input_cat_dict['mass'][self.j] = self.mass
        input_cat_dict['chi2'][self.j] = self.z_phot_chi2
        write_output(input_cat_dict,comments,'test.fits') 
    
    def show_cutout(self):

        cutout_file = specs_for_Ivan_dir+'gds-grizli-v5.0_0'+str(self.ID_here)+'.beams.fits'
        hdu = fits.open(cutout_file)
        header_band = hdu[1].header
        data_band = hdu[1].data 
        hdu.close()
        band_ps = pix_scale(header_band)

        p1 = float(self.le_cut_cutout1.text())
        p2 = float(self.le_cut_cutout2.text())

        # make sure that percentiles work, reset to standard values in case of wrong entry
        if p1<0:
            p1 = 0
        if p2>100:
            p2 = 100
        if p1>100:
            p1 = 0
        if p2<0:
            p2 = 100
        if p2<p1:
            p1 = 0
            p2 = 100

        self.perc10 = np.percentile(data_band,p1)
        self.perc90 = np.percentile(data_band,p2)
        
        xy_band = radec_to_pix(input_cat_dict['ra'][self.j],
                               input_cat_dict['dec'][self.j],header_band)

        x_band = xy_band[0]
        y_band = xy_band[1]
        self.cutout.clear()
        img_band = pg.ImageItem(border='k')
        img_band.setLookupTable(viridis)
        data_band_flipped = np.rot90(np.fliplr(data_band))
        img_band.setImage(data_band_flipped)
        
        self.cut_cutout1 = self.perc10 #float(self.le_cut_cutout1.text()) * 10**-23
        self.cut_cutout2 = self.perc90 #float(self.le_cut_cutout2.text()) * 10**-23
        cut_cutout1[0] = self.cut_cutout1
        cut_cutout2[0] = self.cut_cutout2

        img_band.setLevels([cut_cutout1[0],cut_cutout2[0]]) 
        self.view_cutout = self.cutout.addViewBox()
        self.view_cutout.addItem(img_band)


        #width_window_pix = width_window[0] / band_ps
        #self.view_cutout.setRange(QtCore.QRectF(x_band-width_window_pix/2.,
        #                                     y_band-width_window_pix/2.,
        #                                     width_window_pix,
        #                                     width_window_pix))
        self.view_cutout.setAspectLocked(True)


        ### TODO: position of lines seems shifted and doesn't match with catalogue

        #test_line = pg.InfiniteLine(angle=90,movable=False,
        #                            pen=pg.mkPen(color=c_data_crossline, width = 1))
        #test_line.setPos([x_band,0]) 
        #test_line2 = pg.InfiniteLine(angle=0,movable=False,
        #                             pen=pg.mkPen(color=c_data_crossline, width = 1))                          
        #test_line2.setPos([0,y_band]) 
        #self.view_cutout.addItem(test_line)
        #self.view_cutout.addItem(test_line2)
        
    def run_eazy(self):

        if self.z_slider>0:
            eazy_inst.ZSPEC[self.j] = np.round((self.z_slider/1000.),3)

        #actual fit
        with suppress_stdout():
            eazy_inst.fit_catalog(idx = np.asarray([self.j]))

        self.status.showMessage('Getting eazy outputs...')

        #get output
        with suppress_stdout():     
            self.zout, _ = eazy_inst.standard_output()   

        ### move all eazy output in directory
        here_files = os.listdir('.')
        for file in here_files:
            if 'output' in file and not os.path.isdir(file):
                shutil.move(file, 'eazy_output/'+file)

        self.status.showMessage('Finished with eazy...')

    def show_eazy_fig(self):

        self.run_eazy()

        with suppress_stdout():
            eazy_fig, eazy_ax = eazy_inst.show_fit(self.zout['id'][self.j], show_fnu=1, show_components=False)
            plt.close('all')

        self.eazy_fig_widget.clear()
        pfs = self.eazy_fig_widget.addPlot()

        ind = self.j
        eazy_id = str(eazy_inst.OBJID[ind])
        eazy_z = eazy_inst.zbest[ind]

        pfs.setTitle('z = '+str(np.round(eazy_z,2))+', ID='+str(ID_phot_cat[self.j]))

        # these are matplotlib.patch.Patch properties
        props = dict(boxstyle='round', facecolor='white',edgecolor='k', alpha=0.5)
        textstr = '\n'+r'$z=%.2f$' % (eazy_z)
        #pfs.text(1.02*10**3, 0.27, textstr, fontsize=14)

        lam1 = eazy_ax['pivot']
        lam2 = eazy_ax['templz']#/10000
        valid = eazy_ax['valid']
        invalid = valid==False

        err_pen = {'color': 0.8, 'width': 10, 'alpha':0.5}
        error1 = pg.ErrorBarItem(x=np.log10(lam1[valid]),y=eazy_ax['model'][valid],top=eazy_ax['emodel'][valid],bottom=eazy_ax['emodel'][valid],pen=err_pen)
        pfs.addItem(error1)
        error2 = pg.ErrorBarItem(x=np.log10(lam1[valid]),y=eazy_ax['fobs'][valid],top=eazy_ax['efobs'][valid],bottom=eazy_ax['efobs'][valid],pen=err_pen)
        pfs.addItem(error2)
        err_pen3 = {'color': 0.95, 'width': 5, 'alpha':0.5}
        error3 = pg.ErrorBarItem(x=np.log10(lam1[invalid]),y=eazy_ax['fobs'][invalid],top=eazy_ax['efobs'][invalid],bottom=eazy_ax['efobs'][invalid],pen=err_pen3)
        pfs.addItem(error3)

        scatter1 = pg.ScatterPlotItem(size=15, brush=pg.mkBrush(np.asarray(cl.viridis_all[3])*255))
        scatter1.setData(np.log10(lam1[valid]), eazy_ax['model'][valid])
        pfs.addItem(scatter1)

        scatter2 = pg.ScatterPlotItem(size=15, brush=pg.mkBrush(np.asarray(cl.viridis_all[1])*255))
        scatter2.setData(np.log10(lam1[valid]), eazy_ax['fobs'][valid])
        pfs.addItem(scatter2)

        scatter3 = pg.ScatterPlotItem(size=15, symbol='x', brush=pg.mkBrush(0.95),alpha=0.5)
        scatter3.setData(np.log10(lam1[invalid]), eazy_ax['fobs'][invalid])
        pfs.addItem(scatter3)

        model_brush = pg.mkBrush(np.asarray(cl.viridis_all[1])*255)

        pfs.plot(lam2,eazy_ax['templf'],color=cl.viridis_all[3],alpha=1)

        styles = {'color':'r', 'font-size':'20px'}
        pfs.setLabel('bottom','<p>λ<sub>obs</sub> [μm]</p>', **styles)
        pfs.setLabel('left','<p>f<sub>ν</sub> [μJy]</p>', **styles)
        pfs.setLogMode(True,False)
        pfs.addLegend()
        #pfs.showGrid(x=True, y=True)
        #pfs.setXRange(0.1,10)
        pfs.setXRange(3.5,4.8)
        pfs.setYRange(-0.1,0.3)

        self.eazy_z_widget.clear()
        pfs_z = self.eazy_z_widget.addPlot()

        zgrid = eazy_inst.zgrid
        lnp_i = eazy_inst.lnp[eazy_ax['ix'],:]
        pz = np.exp(lnp_i).flatten()

        pfs_z.plot(zgrid, pz, color='orange', label=None)
        #pfs_z.fill_between(zgrid, pz, pz*0, color='yellow', alpha=0.5,label=None)

        phigh = pg.PlotCurveItem(zgrid, pz, pen = 'k')           
        plow = pg.PlotCurveItem(zgrid, pz*0, pen = 'k')                  
        pfill = pg.FillBetweenItem(phigh, plow, brush='y')
        pfs_z.addItem(phigh)
        pfs_z.addItem(plow)
        pfs_z.addItem(pfill)

        pfs_z.setLabel('bottom','z', **styles) 
        pfs_z.setLabel('left','p(z)', **styles)
        pfs_z.showGrid(x=True, y=True)
        #pfs_z.set_yticklabels([])
        pfs_z.setXRange(0,10)

def main():
    app = QApplication(sys.argv)
    main = main_Window()
    main.show()    
    app.exec_()

if __name__ == '__main__':
    main()



AttributeError: module 'pyqtgraph.Qt.QtGui' has no attribute 'QGridLayout'